In [48]:
import configparser
#from six.moves import cPickle
#import glob
import pandas as pd
#import glob
#import numpy as np
#import os
#import theano
#import theano.gpuarray
#theano.gpuarray.use("cuda" + str(0))
#import matplotlib.pyplot as plt
from nolearn.lasagne import visualize
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import theano
import theano.gpuarray
import pygpu
from pygpu import gpuarray
theano.gpuarray.use("cuda"+str(0))
from six.moves import cPickle
import os
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import statistics as stats
import glob
from texttable import Texttable

config = configparser.ConfigParser()
config.read('config.ini')

def hdmdt(mjd,mag):
    (smjd,smag) = list(zip(*sorted(list(zip(mjd,mag)))))
    #dmdt=np.zeros(shape=(len(dmints)-1,len(dtints)-1))
    maxpts = (len(mjd)*(len(mjd)-1))/2
    dmjd = []
    dmag = []

    # generate differences (w.r.t. time and mags)
    for i in range(len(mjd)):
        for j in range(i+1,len(mjd)):
            dmjd.append(mjd[j]-mjd[i])
            dmag.append(mag[j]-mag[i])

    # sort w.r.t. to first component (dmjd)
    (sdmjd,sdmag) = list(zip(*sorted(list(zip(dmjd,dmag)))))

    h=plt.hist2d(sdmjd,sdmag,bins=[dtints,dmints])
    dmdt=h[0]
    dmdt=np.transpose(dmdt)
    dmdt=(maxval*dmdt/maxpts+0.99999).astype(int)
    return dmdt

def predict(x,out):
    
    X_test=np.stack(x)
    (ff,gg,hh)=X_test.shape
    X=X_test.reshape(ff,1,gg,hh)
    preds_prob=net.predict_proba(X)
    preds=net.predict(X)

    count=[len(np.where(preds==classes[i])[0]) for i in range(len(classes))]
    percent=[(count[i]/sum(count))*100.0 for i in range(len(classes))]

    df_res=pd.DataFrame(np.transpose(np.array([count,percent])), columns=['Nos','% of dataset'], index=classes)

    df_res.to_csv(out)
    


['config.ini']

In [8]:
f=open(config['CNNs']['Model'],'rb')
net=cPickle.load(f)
f.close()

dmints=config['data']['dmints'].split(',')
dmints=[float(dmints[i]) for i in range(len(dmints))]

dtints=config['data']['dtints'].split(',')
dtints=[float(dtints[i]) for i in range(len(dtints))]

classes=config['data']['classes'].split(',')
classes=[int(classes[i]) for i in range(len(classes))]

l=glob.glob(config['data']['filename'])
dataset=config['data']['dataset']

if dataset=='ATLAS':
    x_c=[]
    x_o=[]
    for q in range(len(l)):
        name=l[q].split(".")[0].split("/")[1] #str(l[q])[:-4]
        df=pd.read_csv(l[q])
        lcid=sorted(set(df['objid']))
        for ii in range(len(lcid)):
            df1=df[df['objid']==lcid[ii]]
            df2=df1.loc[df['filter']=='c']
            df22=df1.loc[df['filter']=='o']
            if len(df2)>=2:
                df3=df2[['objid','mjd','m']]
                mjd=pd.Series.as_matrix(df3['mjd'])
                mag=pd.Series.as_matrix(df3['m'])
                dmdt=hdmdt(mjd,mag)    
                x_c.append(dmdt)
            
            if len(df22)>=2:
                df33=df22[['objid','mjd','m']]
                mjd=pd.Series.as_matrix(df33['mjd'])
                mag=pd.Series.as_matrix(df33['m'])
                dmdt=hdmdt(mjd,mag)
                x_o.append(dmdt)

    predict(x_c,config['output']['filename']+"_c.csv")
    predict(x_o,config['output']['filename']+"_o.csv")
    
if dataset=="PTF":

    x=[]

    for q in range(len(l)):
        df=pd.read_table(l[q],header=None,delim_whitespace=True)
        mjd=pd.Series.as_matrix(df[0])
        mag=pd.Series.as_matrix(df[1])
        lcid=list(set(pd.Series.as_matrix(df[5])))[0]
        dmdt=hdmdt(mjd,mag)
        x.append(dmdt)
    
    predict(x,config['output']['filename']+".csv")

if dataset=="Gaia":
    
    x_g=[]
    x_rp=[]
    x_bp=[]
    
    for q in range(len(l)):
        name=l[q].split(".")[0].split("/")[1] #str(l[q])[:-4]
        df1=pd.read_csv(l[q])
        df2=df1.loc[df['band']=='G']
        df22=df1.loc[df['band']=='RP']
        df222=df1.loc[df['band']=='BP']
        
        if len(df2)>=2:
            mjd=pd.Series.as_matrix(df2['time'])
            mag=pd.Series.as_matrix(df2['mag'])
            dmdt=hdmdt(mjd,mag)
            x_g.append(dmdt)
            
        if len(df22)>=2:
            mjd=pd.Series.as_matrix(df22['time'])
            mag=pd.Series.as_matrix(df22['mag'])
            dmdt=hdmdt(mjd,mag)
            x_rp.append(dmdt)
            
        if len(df222)>=2:
            mjd=pd.Series.as_matrix(df222['time'])
            mag=pd.Series.as_matrix(df222['mag'])
            dmdt=hdmdt(mjd,mag)
            x_bp.append(dmdt)

    predict(x_g,config['output']['filename']+"_g.csv")
    predict(x_rp,config['output']['filename']+"_rp.csv")
    predict(x_bp,config['output']['filename']+"_bp.csv")
            
            
            

            

FileNotFoundError: [Errno 2] No such file or directory: 'p_obj.save'